In [1]:
"""
Created By Kevin Du for OSGC Scholarship Program. Credits to "Neural network powered COVID-19 spread forecasting model"\
by Michał Wieczorek, Jakub Siłka, Marcin Woźniak for architecture and data processing.

https://doi.org/10.1016/j.chaos.2020.110203

~~notes notes notes
Purpose:
Intended result:

"""

'\nCreated By Kevin Du for OSGC Scholarship Program. Credits to "Neural network powered COVID-19 spread forecasting model"by Michał Wieczorek, Jakub Siłka, Marcin Woźniak for architecture and data processing.\n\nhttps://doi.org/10.1016/j.chaos.2020.110203\n\n~~notes notes notes\nPurpose:\nIntended result:\n\n'

In [2]:
import tensorflow as tf
from keras import layers
import math
import numpy as np
import pandas as pd
import csv
import pprint

In [70]:
##---------Import of data----------------

#State/territory list (sorted alphabetically)
statelist= [[] for _ in range(58)]

#List of non-State and non-DC rows (start counting at 1)
staterowlist=[4,11,15,16,41,46,54]

#folderpath to 
folderpath = r"C:\Kevin\Applications\OSGC Undergraduate scholarship\COVID-19\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports_us" + "\\"

with open(r"C:\Kevin\Applications\OSGC Undergraduate scholarship\Date List.csv") as file:
    reader = csv.reader(file)
    datelist = list(reader)
datelist=datelist[0]

#total count of days of data
count=len(datelist)

for i in range(0, count):
    splitdate = datelist[i].split("/")
    datelist[i] = splitdate[0] + "-" + splitdate[1] + "-20" + splitdate[2]
#print(datelist)

#could optimize by adding the 'columns' as a row, then flipping everything with numpy

#extract all the confirmed cases by date from all the data
for date in datelist:
    filepath = folderpath + date + ".csv"
    with open(filepath) as file:
        reader = csv.reader(file)
        data = list(reader)
    for i in range(58):
        statelist[i].append(data[i+1][5])
    
#Infection rates (Total cases) now stored in an array, with each row representing a state or region,
#and each column representing a date for the infection
    
    
#--------------Normalization of data------------   

#Make numpy recognize that data is numerical
statelist = np.array(statelist).astype(float)

#Normalize by the highest infection rate
#should maybe normalize by total population
maxstatelist = np.amax(statelist, axis=1)
statelist = statelist / maxstatelist[:,None]

#Infection rates now in terms of (cases/max one day infection cases)


#---------------Formatting of data---------------

#number of days in a grouping
time_step = 14

#grouping of data
gstatelist=[]

# group data into a list of first by start date, then by state. 
# Ex. The first 58 column arrays are 
for i in range(time_step, count-1):
    for j in range(58):
        gstatelist.append(statelist[j,(i-time_step):i].tolist())
        
xtrain_set=gstatelist[:-90*58]
xvalidation_set=gstatelist[-90*58:]

ytrain_set=statelist[:,time_step+1:-90].flatten().tolist()
yvalidation_set=statelist[:,-90:].flatten().tolist()

#Test print lines to be removed when finished with first draft
#print('shape of gstatelist is '+ str(len(gstatelist)) +', '+str(len(gstatelist[0])))
#print('length of xtrain is '+ str(len(xtrain_set)))
#print('length of ytrain is '+str(len(ytrain_set)))
#print('length of xval is '+ str(len(xvalidation_set)))
#print('length of yval is '+ str(len(yvalidation_set)))

#Infection rates (input set) are now in sets of 14 days, with the last three months of data reserved for validation 
#(testing set will come from most recent data not included)
#

In [78]:
#hyperparameters
#input_shape=(14,)
#neuron



model = tf.keras.models.Sequential()
model.add(layers.Dense(units = 256, activation = 'tanh', input_shape = (14,)))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(units = 96, activation = 'tanh'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(units = 48, activation = 'tanh'))
model.add(layers.Dense(units = 28, activation = 'tanh'))
model.add(layers.Dense(units = 48, activation = 'tanh'))
model.add(layers.Dense(units = 12, activation = 'tanh'))
model.add(layers.Dense(units = 6, activation = 'tanh'))
model.add(layers.Dense(units = 1, activation = 'relu'))
#need to find a better loss and metric for compiler for covid data- paper doesn't specify much past what is here
model.compile(optimizer = 'Nadam', loss = 'mean_squared_error')




In [79]:

model.fit(xtrain_set, ytrain_set, epochs =1)
test_loss = model.evaluate(xvalidation_set, yvalidation_set)
print(test_loss)

164/164 [==============================] - 0s 986us/step - loss: 0.2312
0.2312467247247696
